# MLEM2 Algorithm

In [1]:
import time
import pandas as pd
import numpy as np
from pandas.api.types import is_numeric_dtype
from itertools import groupby

In [2]:
df = pd.read_csv('../data/trip_data.csv')
# df = pd.read_csv('../data/Echocardiogram.csv')

In [3]:
df = df.drop(['Case'],axis=1)

In [4]:
df

,Wind,Humidity,Temperature,Trip
0,4,low,medium,yes
1,8,low,low,yes
2,4,medium,medium,yes
3,8,medium,high,maybe
4,12,low,medium,maybe
5,16,high,low,no
6,30,high,high,no
7,12,high,high,no


#### --------------------------Defining Goals--------------------------

In [5]:
df_headers = list(df)
concept = df_headers[-1]
concept

'Trip'

In [6]:
#all unique concepts
concept_list = df[concept].unique()
concept_list

array(['yes', 'maybe', 'no'], dtype=object)

In [7]:
#calculating cases by concepts and making sets
temp_list = []
goal_list = []
for item in concept_list:
    for index, row in df.iterrows():
        if row[concept] == item:
            temp_list.append(index+1)
    goal_list.append(temp_list)
    temp_list = []

In [8]:
goal_list

[[1, 2, 3], [4, 5], [6, 7, 8]]

#### -------------------Finding numeric column---------------------

In [9]:
# numeric_col = df.dtypes[df.dtypes != "object"].index[0]
numeric_col = df.select_dtypes(include=[np.number]).columns.tolist()

In [10]:
numeric_col

['Wind']

In [11]:
len(numeric_col)

1

### Discretization of Numeric Attributes !!

In [12]:
case_list = []

In [13]:
def discretize(numeric_col):
    
    #Sorting the values of numeric column
    sort_col = df[numeric_col].sort_values()
    df['sort_col'] = sort_col.values
    point_list = df['sort_col'].unique()
    
    #Finding average between each two points
    avg_list = []
    for i in range(len(point_list)-1):
        avg = (point_list[i] + point_list[i+1])/2
        avg_list.append(int(avg))
        
    #Performing the discretization and adding the cases
    for i in avg_list:
        case = str(numeric_col) + "," + str(point_list[0]) + ".." + str(i)
        case2 = str(numeric_col) + "," + str(i) + ".." + str(point_list[len(point_list)-1])
        case_list.append(case)
        case_list.append(case2)

In [14]:
#Follow the first route if n. of numeric attributes is 1, otherwise follow the second route
if len(numeric_col) > 1:
    for item in numeric_col:
        discretize(item)
else:
    discretize(numeric_col[0])

In [15]:
case_list

['Wind,4..6',
 'Wind,6..30',
 'Wind,4..10',
 'Wind,10..30',
 'Wind,4..14',
 'Wind,14..30',
 'Wind,4..23',
 'Wind,23..30']

In [16]:
attributes = list(df)
attributes

['Wind', 'Humidity', 'Temperature', 'Trip', 'sort_col']

In [17]:
#Loop through all the attributes except last 2 - Concept and sort_col
for item in attributes[:-2]:
    print(item)
    #check for non numeric columns
    if not is_numeric_dtype(df[item]):
        temp = df[item].unique()
        for i in temp:
            case = item + "," + i
            case_list.append(case)
        

Wind
Humidity
Temperature


In [18]:
#This is the final list of cases after discretization
case_list

['Wind,4..6',
 'Wind,6..30',
 'Wind,4..10',
 'Wind,10..30',
 'Wind,4..14',
 'Wind,14..30',
 'Wind,4..23',
 'Wind,23..30',
 'Humidity,low',
 'Humidity,medium',
 'Humidity,high',
 'Temperature,medium',
 'Temperature,low',
 'Temperature,high']

#### ---------------Building (a,v) pairs---------------------

In [19]:
temp_list = []
att_val_list = []
for item in case_list:
    a,b = item.split(",") #a = attribute and b = value
    if "." in b:
        start,end = b.split("..")
        for index, row in df.iterrows():
            if row[a] >= int(start) and row[a] <= int(end):
                temp_list.append(index+1)
        print(temp_list)
        att_val_list.append(temp_list)
        temp_list = []
        
    else:
        for index, row in df.iterrows():
            if row[a] == b:
                temp_list.append(index+1)
        print(temp_list)
        att_val_list.append(temp_list)
        temp_list = []

[1, 3]
[2, 4, 5, 6, 7, 8]
[1, 2, 3, 4]
[5, 6, 7, 8]
[1, 2, 3, 4, 5, 8]
[6, 7]
[1, 2, 3, 4, 5, 6, 8]
[7]
[1, 2, 5]
[3, 4]
[6, 7, 8]
[1, 3, 5]
[2, 6]
[4, 7, 8]


In [20]:
att_val_list

[[1, 3],
 [2, 4, 5, 6, 7, 8],
 [1, 2, 3, 4],
 [5, 6, 7, 8],
 [1, 2, 3, 4, 5, 8],
 [6, 7],
 [1, 2, 3, 4, 5, 6, 8],
 [7],
 [1, 2, 5],
 [3, 4],
 [6, 7, 8],
 [1, 3, 5],
 [2, 6],
 [4, 7, 8]]

In [21]:
#Creating data for case and att-value list
data = {'Cases': case_list, 'att_val': att_val_list}

In [22]:
df2 = pd.DataFrame(data)

In [23]:
#Cases and corresponding att-value pairs
df2

,Cases,att_val
0,"Wind,4..6","[1, 3]"
1,"Wind,6..30","[2, 4, 5, 6, 7, 8]"
2,"Wind,4..10","[1, 2, 3, 4]"
3,"Wind,10..30","[5, 6, 7, 8]"
4,"Wind,4..14","[1, 2, 3, 4, 5, 8]"
5,"Wind,14..30","[6, 7]"
6,"Wind,4..23","[1, 2, 3, 4, 5, 6, 8]"
7,"Wind,23..30",[7]
8,"Humidity,low","[1, 2, 5]"
9,"Humidity,medium","[3, 4]"


In [24]:
df3 = df2

### --------------Developing MLEM2 Algorithm--------------------
#### Find Goal Intersect Pairs !!

In [25]:
def findGoalIntersect(goal):
    goalIntersect = []
    
    for index, row in df3.iterrows():
        #List containing intersection of (a,v) pairs and goal
        goalIntersect.append(set(row['att_val']).intersection(set(goal)))
          
    #Check if goal_intersect column exists
    if 'goal_intersect' in df3:
        df3['goal_intersect'] = goalIntersect
    else:
        #Insert new column with the recent iteration
        df3.insert(2, 'goal_intersect', goalIntersect)

#### Select the Proper Case covering most part of the goal and having least no. of elements !!

In [26]:
def findCases(df3):
    #Find the cases with maximum goal coverage
    m = max(df3['goal_intersect'], key=len)
    possible_cases = [i for i, j in enumerate(df3['goal_intersect'].tolist()) if j == m]
    
    #Index of the case covering max goal and having min no. of elements
    new_df = df3.iloc[possible_cases,:]
    m1 = min(new_df['att_val'], key=len)
    final_case = [i for i, j in enumerate(df3['att_val'].tolist()) if j == m1]
    
    return final_case[0]

#### Combine the Numeric Interval in the conditions if overlapped !!

In [27]:
def combineInterval(test_condition):
    
    test_num = [] #This will contain the conditions having interval
    test_str = [] #This will contain the conditions having no interval
    
    #Loop through to seprate contions having intervals and no intervals
    for item in test_condition:
        if ".." in item:
            test_num.append(item)
        else:
            test_str.append(item)
   
    #Group the conditions having interval based on same attributes
    grouped = [list(g) for k, g in groupby(test_num, lambda s: s.partition(',')[0])]
    
    final_list = []
    
    #Actually combining the intervals
    for list1 in grouped:
        greatest = 0
        smallest = 0
        for item in list1:
            part1,part2 = item.split(",")
            start,stop = part2.split("..")
            start = int(start)
            stop = int(stop)

            if greatest == 0 and smallest == 0:
                greatest = start
                smallest = stop

            if start > greatest:
                greatest = int(start)

            if stop < smallest:
                smallest = int(stop)

        con_tmp = part1 + "," + str(greatest) + ".." + str(smallest)
        final_list.append(con_tmp)  
            
    actual_condition = final_list + test_str
    
    return actual_condition

#### Dropping Conditions Check !!

In [101]:
def dropCondition(condition):
    print (condition)
    for item in condition:
        temp_att_val = []
        temp_cond = condition
        temp_cond.remove(item)
      
        for i in temp_cond:
            location = df3.index[df3['Cases'] == i].tolist()
            element = df3['att_val'].loc[location[0]]
         
            temp_att_val.append(set(element))
            
        print(temp_att_val)
        intersection = set.intersection(*temp_att_val)
    print(intersection)
      

In [102]:
condition2 = ['Wind,10..30', 'Humidity,low', 'Temperature,medium']
dropCondition(condition2)

['Wind,10..30', 'Humidity,low', 'Temperature,medium']
[{1, 2, 5}, {1, 3, 5}]
[{1, 2, 5}]
{1, 2, 5}


#### MLEM2 Algorithm !!

In [29]:
def stepAlgo(df3,selected_case,current_goal,B,condition,concept_curr):
    
    rule_set = []
    
    while current_goal != set():

        #Check if the selected case is a subset of the current goal

        #List of current case
        A = df3['att_val'].loc[selected_case] 

        if B == set():
            #Copy over the current set elements to B
            for i in range(len(A)):
                B.add(A[i])

        #Elements of intersection of current and previous set
        A = set(A).intersection(B)
        B = A

        #Check if intersecting elements are subset of Goal
        if A.issubset(current_goal):
            #Current goal is updated after discarding the already covered goal by new rule
            current_goal = set(current_goal) - A

            #Extract the current case
            curr_case = df3['Cases'].loc[selected_case]
            #Add the conditions of a Rule
            condition.append(curr_case)
            
            #Check for possibility of dropping conditions
#             dropCondition(condition)
            
            #Combine the interval
            condition = combineInterval(condition)

            #Join conditions
            cond = ""
            for item in condition:
                cond = cond + "(" + str(item) + ")" + " & "

            cond = cond[:-2] + "->"
            rule = cond + " (" + concept + "," + concept_curr + ")"
            rule_set.append(rule)

            #Reset everythng and continue for covering rest of the goal
            condition = []
            B = set()
            findGoalIntersect(current_goal)
            selected_case = findCases(df3)


        #If not a subset of current goal
        else:
            #Assign empty set for the selected case for next iteration
            df3['goal_intersect'].loc[selected_case] = set()
            #Extract the current case
            curr_case = df3['Cases'].loc[selected_case]
            #Add the case to the condition list
            condition.append(curr_case)

            #Check for Range overlapping of the remaining cases
            if "." in curr_case:
                second_part = curr_case.split(',')[1]
                start = int(second_part.split('..')[0])
                end = int(second_part.split('..')[1])
                for index, row in df3.iterrows():
                    if "." in row['Cases']:
                        part2 = row['Cases'].split(',')[1]
                        start1 = int(part2.split('..')[0])
                        end1 = int(part2.split('..')[1])

                        #Assign blank set for cases with overlapping ranges
                        if set((range(start,end))).issubset(range(start1,end1)) == True:
                            row['goal_intersect'] = set()

            selected_case = findCases(df3)
                        
    return rule_set

In [30]:
#concept_list and goal_list has 1:1 mapping
final_rules = []
start_time = time.time()

#Running algorithm for all the goals/concepts
for i in range(0,len(goal_list)):
    findGoalIntersect(goal_list[i])
    condition = []
    B = set()
    selected_case = findCases(df3)
    
    rule_set = stepAlgo(df3,selected_case,goal_list[i],B,condition,concept_list[i])
    final_rules.append(rule_set)
    
elapsed_time = time.time() - start_time

In [31]:
print("Time to run the algorithm: ", round(elapsed_time, 3), "Sec")
print("\n")
print(*final_rules, sep='\n')

Time to run the algorithm:  0.037 Sec


['(Wind,4..6) -> (Trip,yes)', '(Temperature,low) & (Humidity,low) -> (Trip,yes)']
['(Wind,6..14) & (Humidity,medium) -> (Trip,maybe)', '(Wind,10..30) & (Humidity,low) & (Temperature,medium) -> (Trip,maybe)']
['(Humidity,high) -> (Trip,no)']


In [32]:
#Order of functions to run for algorithm (after attribute-value pairs are made):
# 1. findGoalIntersect()
# 2. findCases()
# 3. stepAlgo()

In [33]:
# Need to implement dropping condition